# Estimating biomarker ordering

>The sampler for the biomarker ordering can be a bit tricker. The simplest way to do it might be to do a Metropolis-Hastings step where you select two indicies and propose swapping their order. Then you can work out the relative probabilities, evaluate and then accept/reject based on that. It's not the fastest sampler, but it is a lot more straightforward than some ways of doing it.  

In the following, we assume we know the actual $\theta$ and $\phi$ values. Other than those, we know nothing except for participants' observed biomarker values. And we want to estimate the current order in which different biomarkers are affected by the disease in question. 

In [1]:
import pandas as pd 
import numpy as np 
import re 
import altair as alt 
import matplotlib.pyplot as plt 
from collections import Counter

We only have three columns: biomarker, participant, and measurement. 

In [2]:
data = pd.read_csv('data/participant_data.csv')
data.Biomarker = [re.sub("Biomarker ", "", text) for text in data.Biomarker.tolist()]
data_we_have = data.drop(['k_j', 'S_n', 'affected_or_not'], axis = 1)
data_we_have.head()

,Biomarker,participant,measurement
0,0,0,38.410004
1,0,1,35.995276
2,0,2,1.148822
3,0,3,0.791957
4,0,4,0.918809


In [3]:
theta_phi = pd.read_csv('data/means_vars.csv')
theta_phi.head()

,biomarker,theta_mean,theta_var,phi_mean,phi_var
0,0,1.0,0.3,32.0,6.3
1,1,3.0,0.5,31.0,7.4
2,2,5.0,0.2,34.0,9.4
3,3,6.0,1.3,36.0,4.9
4,4,8.0,3.3,38.0,2.5


In [4]:
type(theta_phi['biomarker'][0])

numpy.int64

In [5]:
def fill_up_pdata(pdata, k_j):
    '''Fill up participant data using k_j; basically add two columns: k_j and affected
    Note that this function assumes that pdata already has the S_n column
    Input:
    - pdata: a dataframe of ten biomarker values for a specific participant 
    - k_j: a scalar
    '''
    data = pdata.copy()
    data['k_j'] = k_j
    data['affected'] = data.apply(lambda row: row.k_j >= row.S_n, axis = 1)
    return data 

In [6]:
def compute_single_measurement_likelihood(theta_phi, biomarker, affected, measurement):
    '''Computes the likelihood of the measurement value of a single biomarker
    We know the normal distribution defined by either theta or phi
    and we know the measurement. This will give us the probability
    of this given measurement value. 

    Note that because the likelihood tends to be very very small, 
    we take the natural log of it

    input:
    - theta_phi: the dataframe containing theta and phi values for each biomarker
    - biomarker: an integer between 0 and 9 
    - affected: boolean 
    - measurement: the observed value for a biomarker in a specific participant

    output: a scalar
    '''
    biomarker_params = theta_phi[theta_phi.biomarker == biomarker].reset_index()
    mu = biomarker_params['theta_mean'][0] if affected else biomarker_params['phi_mean'][0]
    var = biomarker_params['theta_var'][0] if affected else biomarker_params['phi_var'][0]
    # sigma = np.sqrt(var)
    likelihood = np.exp(-(measurement - mu)**2/(2*var))/np.sqrt(2*np.pi*var)
    return likelihood

In [7]:
def compute_likelihood(pdata, k_j, theta_phi):
    '''This implementes the formula of https://ebm-book2.vercel.app/distributions.html#known-k-j
    This function computes the likelihood of seeing this sequence of biomarker values for a specific participant
    '''
    data = fill_up_pdata(pdata, k_j)
    likelihood = 1
    for i, row in data.iterrows():
        biomarker = int(row['Biomarker'])
        measurement = row['measurement']
        affected = row['affected']
        likelihood *= compute_single_measurement_likelihood(
            theta_phi, biomarker, affected, measurement)
    return likelihood

## Testing

The above functions can compute the likelihood of a participant's sequence of biomarker data, given that we know the exact ordering and we assume a `k_j`. Next, we will test those functions by selecting a specific participant. We compute the likelihood by trying all possible `k_j` and see whether the one with the highest likelihood is the real `k_j` in the original data. 

In [8]:
p = 15 # we chose this participant
pdata = data[data.participant == p].reset_index(drop=True)
pdata

,Biomarker,participant,measurement,k_j,S_n,affected_or_not
0,0,15,0.826103,8,8,affected
1,1,15,2.045123,8,1,affected
2,2,15,25.286707,8,9,not_affected
3,3,15,7.520045,8,4,affected
4,4,15,6.774157,8,2,affected
5,5,15,-4.433647,8,3,affected
6,6,15,2.173509,8,7,affected
7,7,15,36.169561,8,10,not_affected
8,8,15,7.334693,8,6,affected
9,9,15,8.378574,8,5,affected


In [9]:
# ordering of biomarker affected by the disease
# biomarker: disease stage
# note that the value >= 1
real_ordering_dic = dict(zip(np.arange(10), pdata.S_n))
real_ordering_dic

{0: 8, 1: 1, 2: 9, 3: 4, 4: 2, 5: 3, 6: 7, 7: 10, 8: 6, 9: 5}

In [10]:
# get the participant data without k_j, and affected or not
pdata = data_we_have[data_we_have.participant == p].reset_index(drop=True)
# obtain real ordering:
pdata['S_n'] = pdata.apply(lambda row: real_ordering_dic[int(row['Biomarker'])], axis = 1)
pdata

,Biomarker,participant,measurement,S_n
0,0,15,0.826103,8
1,1,15,2.045123,1
2,2,15,25.286707,9
3,3,15,7.520045,4
4,4,15,6.774157,2
5,5,15,-4.433647,3
6,6,15,2.173509,7
7,7,15,36.169561,10
8,8,15,7.334693,6
9,9,15,8.378574,5


In [11]:
num_biomarkers = len(pdata.Biomarker.unique())
# calculate likelihood for all possible k_j
likelihood_list = [
    compute_likelihood(pdata=pdata, k_j=x, theta_phi=theta_phi) for x in range(num_biomarkers+1)]
kjs = np.arange(11)
dic = dict(zip(kjs, likelihood_list))
df = pd.DataFrame.from_dict(dic, orient='index', columns=['likelihood']).reset_index()
df.sort_values('likelihood', ascending=False)

,index,likelihood
8,8,1.907737e-11
7,7,1.396579e-45
6,6,2.124019e-71
5,5,1.143365e-92
4,4,3.579283e-152
3,3,5.090393e-188
2,2,1.003539e-255
0,0,0.000000e+00
1,1,0.000000e+00
9,9,0.000000e+00


<!-- From the above result, we can see that the most likelihood `k_j` is 8, which is in fact the real `k_j` in the participant data.  -->

## Metropolis-Hastings Algorithm Implementation

Next, we will implement the metropolis-hastings algorithm using the above functions. 

In [12]:
def average_all_likelihood(pdata, num_biomarkers, theta_phi):
    '''This is to compute https://ebm-book2.vercel.app/distributions.html#unknown-k-j
    '''
    return np.mean([compute_likelihood(pdata=pdata, k_j=x, theta_phi=theta_phi) for x in range(num_biomarkers+1)])

In [13]:
def compute_ln_likelihood_assuming_ordering(ordering, data, num_participants, num_biomarkers, theta_phi):
    """Compute the (ln version of) the likelihood of seeing all participants' data,
    assuming that we already know the ordering
    Inputs:
        - ordering: an array of ordering for biomarker 0-9
        - data: data_we_have
        - num_participants
        - num_biomarkers 
    Outputs:
        - ln(likelihood)
    """
    # biomarker - order dict
    ordering_dic = dict(zip(np.arange(num_biomarkers), ordering))
    # fill up S_n column using the ordering dict
    # copy first in order not to change data_we_have
    filled_data = data.copy()
    filled_data['S_n'] = filled_data.apply(lambda row: ordering_dic[int(row['Biomarker'])], axis = 1)
    ln_likelihood = 0 
    for p in range(num_participants):
        pdata = filled_data[filled_data.participant == p].reset_index(drop=True)
        average_likelihood = average_all_likelihood(pdata, num_biomarkers, theta_phi)
        p_ln_likelihood = (
            # natural logarithm
           np.log(average_likelihood) 
           if average_likelihood > 0
           # this is to avoid np.log(0)
           else np.log(average_likelihood + 1e-20)
        )
        ln_likelihood += p_ln_likelihood
    return ln_likelihood

A brush up on log:

$L = P1 \cdot P2 \cdot P3$

$\ln(L) = \ln (P1 \cdot P2 \cdot P3) = \ln(P1) + \ln(P2) + \ln(P3)$

You may wonder why above I am using the second equation to get the likelihood of observing a specific participant's data. This is because if we want to estimate $k_j$, we need to know $S$, which we do not know and are trying to estimate here. 

If we want to estimate $S$ through using the first equation, we need $k_j$, but we do not know it. 

To sum up, to obtain $S$, we need to know $k_j$. And to obtain $k_j$, we need to know $S$. Therefore, we are not able to use the first equation. 

In [14]:
def metropolis_hastings(data, iterations, theta_phi):
    '''Implement the metropolis-hastings algorithm
    Inputs: 
        - data: data_we_have
        - iterations: number of iterations

    Outputs:
        - best_order: a numpy array
        - best_likelihood: a scalar 
    '''
    num_participants = len(data.participant.unique())
    num_biomarkers = len(data.Biomarker.unique())

    # initialize an ordering and likelihood
    # note that it should be a random permutation of numbers 1-10
    best_order = np.random.permutation(np.arange(1, 11))
    best_likelihood = -np.inf 
    # best_order = np.array(list(real_ordering_dic.values()))
    # best_likelihood = compute_likelihood_based_on_ordering(
    #     best_order, data, num_participants, num_biomarkers, theta_phi
    # )
    for _ in range(iterations):
        new_order = best_order.copy()
        # randomly select two indices
        a, b = np.random.choice(num_biomarkers, 2, replace=False)
        # swapping the order
        new_order[a], new_order[b] = new_order[b], new_order[a]
        ln_likelihood = compute_ln_likelihood_assuming_ordering(
            new_order, data, num_participants, num_biomarkers, theta_phi)
        # if swapping results in higher likelihood, 
        # update the likelihood and accept the new ordering 
        if ln_likelihood > best_likelihood:
            best_likelihood = ln_likelihood 
            best_order = new_order
        else: 
            # ratio = likelihood/best_likelihood
            # np.exp(a - b) = np.exp(a)/np.exp(b)
            acceptance_ratio = np.exp(ln_likelihood - best_likelihood)
            random_number = np.random.rand()
            if random_number < acceptance_ratio:
                best_likelihood = ln_likelihood
                best_order = new_order
        print(f"iteration {_ + 1} done")
    return best_order, best_likelihood


When `ln_likelihood` is smaller than `best_likelihood`, we still conditionally accept `ln_likelihood`. This is "to allow exploration of the parameter space. This step is crucial in MCMC algorithms to avoid getting stuck in local maxima and to ensure that the sampling covers the entire distribution space adequately".

Note that the condition is `if random_number < acceptance_ratio`. This `<` cannot be changed to `>`. Why?

When the `accepatance_ratio` is small, then it indicates `ln_likelihood` is small. We do not want to accept this `ln_likelihood` in this case. Because `ln_likelihood` is small, `random_number < acceptance_ratio` is less likely than `random_number > acceptance_ratio`. That's why we use `<` rather than `>`. 

In [15]:
best_order, best_likelihood = metropolis_hastings(
    data = data_we_have, iterations = 10, theta_phi = theta_phi)

iteration 1 done


iteration 2 done


iteration 3 done


iteration 4 done


iteration 5 done


iteration 6 done


iteration 7 done


iteration 8 done


iteration 9 done


iteration 10 done


In [16]:
best_order, np.array(list(real_ordering_dic.values()))

(array([ 9,  7,  1,  5,  6,  3,  8, 10,  2,  4]),
 array([ 8,  1,  9,  4,  2,  3,  7, 10,  6,  5]))

## Unknown theta and phi

I found it challenging to infer the ordering of biomarkers affected by the disease without knowing theta and phi. This is because we do not need to know participants' real disease stage in the formula of https://ebm-book2.vercel.app/distributions.html#unknown-k-j; However, without knowing all participants' disease stages, we are not able to estimate theta and phi. 

In [17]:
data.head()

,Biomarker,participant,measurement,k_j,S_n,affected_or_not
0,0,0,38.410004,4,8,not_affected
1,0,1,35.995276,7,8,not_affected
2,0,2,1.148822,8,8,affected
3,0,3,0.791957,10,8,affected
4,0,4,0.918809,8,8,affected


In [18]:
actual_stage_dict = dict(zip(data.participant, data.k_j))
actual_stages = np.array(list(actual_stage_dict.values()))
actual_stages

array([ 4,  7,  8, 10,  8, 10,  6,  2,  2,  9,  2,  7,  0,  7,  5,  8,  2,
        1,  2,  0,  4,  4,  7,  8,  1,  3,  4,  7,  3,  7,  2,  8,  5,  5,
        6,  6,  9,  2,  1,  3,  6,  3, 10,  3,  4,  9,  3,  0,  0,  4,  8,
        0,  8,  3,  7,  9,  3,  3,  3,  5,  5,  3,  0,  2,  5, 10,  3,  1,
        3,  2,  5,  3,  1, 10,  6,  2,  6, 10,  3,  2,  3,  7,  5,  0,  1,
        0,  8,  8,  3,  8,  9,  6,  5,  9,  6,  3,  3,  5,  8,  0])

In [19]:
data_we_have.head()

,Biomarker,participant,measurement
0,0,0,38.410004
1,0,1,35.995276
2,0,2,1.148822
3,0,3,0.791957
4,0,4,0.918809


In [20]:
def add_kj_and_affected(data_we_have, participant_stages, num_participants):
    '''This is to fill up data_we_have. 
    Basically, add two columns: k_j, and affected, based on the initial or updated participant_stages
    
    Inputs:
        - data_we_have
        - participant_stages: np array 
        - participants: 0-99
    '''
    participant_stage_dic = dict(zip(np.arange(0,num_participants), participant_stages))
    data_we_have['k_j'] = data_we_have.apply(lambda row: participant_stage_dic[row.participant], axis = 1)
    data_we_have['affected'] = data_we_have.apply(lambda row: row.k_j >= row.S_n, axis = 1)
    return data_we_have 

In [21]:
def estimate_params_exact(m0, n0, s0_sq, v0, data):
    '''This is to estimate means and vars based on conjugate priors
    Inputs:
        - data: a vector of measurements for a specific biomarker in a specific group (affected or not)
        - m0: prior estimate of $\mu$.
        - n0: how strongly is the prior belief in $m_0$ is held.
        - s0_sq: prior estimate of $\sigma^2$.
        - v0: prior degress of freedome, influencing the certainty of $s_0^2$.
    
    Outputs:
        - mu estiate, var estimate
    '''
    # Data summary
    sample_mean = np.mean(data)
    sample_size = len(data)
    sample_var = np.var(data, ddof=1)  # ddof=1 for unbiased estimator

    # Update hyperparameters for the Normal-Inverse Gamma posterior
    updated_m0 = (n0 * m0 + sample_size * sample_mean) / (n0 + sample_size)
    updated_n0 = n0 + sample_size
    updated_v0 = v0 + sample_size 
    updated_s0_sq = (1 / updated_v0) * ((sample_size - 1) * sample_var + v0 * s0_sq + 
                    (n0 * sample_size / updated_n0) * (sample_mean - m0)**2)
    updated_alpha = updated_v0/2
    updated_beta = updated_v0*updated_s0_sq/2

    # Posterior estimates
    mu_posterior_mean = updated_m0
    sigma_squared_posterior_mean = updated_beta/updated_alpha

    return mu_posterior_mean, sigma_squared_posterior_mean

In [22]:
def get_estimated_theta_phi(num_biomarkers, data_we_have):
    '''To get estimated parameters, returns a Pandas DataFrame
    Input:
    - num_biomarkers: 10
    - data_we_have: full data filled up by S_n, k_j and affected

    Output: 
    - estimate_means_vars_df, just like means_vars_df, containing the estimated mean and var for 
      distribution of biomarker values when the biomarker is affected and not affected
    '''
    # empty list of dictionaries to store the estimations
    means_vars_estimate_dict_list = []
    # 0 - 9
    for biomarker in range(num_biomarkers): 
        dic = {'biomarker': biomarker}  # Initialize dictionary outside the inner loop
        for affected in [True, False]:
            data_full = data_we_have[(data_we_have.Biomarker == str(biomarker)) & (
            data_we_have.affected == affected)]
            data = np.array(data_full.measurement)
            if len(data) == 0:
                print(biomarker)
                print(data_full)
            mu_estimate, var_estimate = estimate_params_exact(
                m0 = 0, n0 = 1, s0_sq = 1, v0 = 1, data=data)
            if affected:
                dic['theta_mean'] = mu_estimate
                dic['theta_var'] = var_estimate
            else:
                dic['phi_mean'] = mu_estimate
                dic['phi_var'] = var_estimate
        means_vars_estimate_dict_list.append(dic)
    estimate_theta_phi = pd.DataFrame(means_vars_estimate_dict_list)
    return estimate_theta_phi

In [23]:
def metropolis_hastings_unknown_theta_phi(data_we_have, iterations):
    num_participants = len(data_we_have.participant.unique())
    num_biomarkers = len(data_we_have.Biomarker.unique())

    # initialize an ordering and likelihood
    # note that it should be a random permutation of numbers 1-10
    best_order = np.random.permutation(np.arange(1, 11))
    best_likelihood = -np.inf 

    for _ in range(iterations):
        new_order = best_order.copy()
        # randomly select two indices
        a, b = np.random.choice(num_biomarkers, 2, replace=False)
        # swaping the order
        new_order[a], new_order[b] = new_order[b], new_order[a]

        # likelihood of seeing all participants' data 
        # biomarker - order dict
        ordering_dic = dict(zip(np.arange(num_biomarkers), new_order))
        # fill up S_n column using the ordering dict
        # copy first in order not to change data_we_have
        data = data_we_have.copy()
        # now data_we_have has S_n column
        data['S_n'] = data.apply(lambda row: ordering_dic[int(row['Biomarker'])], axis = 1)

        # initialize participant_stages 
        # note that high should be num_stages + 1; otherwise, no participants will be in the stage of 10
        participant_stages = np.random.randint(low = 0, high = num_biomarkers+1, size = num_participants)

        # add kj and affected based on the initial randomized participant_stages
        data = add_kj_and_affected(data, participant_stages, num_participants)
        # print(data.head())

        # get estimated_theta_phi
        estimated_theta_phi = get_estimated_theta_phi(num_biomarkers, data_we_have=data)

        all_participant_likelihood = 0 
        for p in range(num_participants):
            pdata = data[data.participant == p].reset_index(drop=True)
            # initiaze stage_likelihood
            stage_likelihood = np.zeros(num_biomarkers + 1)
            for k_j in range(num_biomarkers +1):
                # even though data above has everything, it is filled up by random stages
                # we don't like it and want to know the true k_j
                pdata_with_this_kj = fill_up_pdata(pdata=pdata, k_j=k_j)
                # likelihood for this participant to have this sequence of biomarker values
                participant_likelihood = 1
                for i, row in pdata_with_this_kj.iterrows():
                    biomarker = int(row['Biomarker'])
                    measurement = row['measurement']
                    affected = row['affected']
                    participant_likelihood *= compute_single_measurement_likelihood(
                        estimated_theta_phi, biomarker, affected, measurement)
                stage_likelihood[k_j] = participant_likelihood
            likelihood_sum = np.sum(stage_likelihood)
            normalized_stage_likelihood = [l/likelihood_sum for l in stage_likelihood]
            sampled_stage = np.random.choice(np.arange(num_biomarkers + 1), p = normalized_stage_likelihood)
            participant_stages[p] = sampled_stage     

            # formula here: https://ebm-book2.vercel.app/distributions.html#unknown-k-j
            average_likelihood = np.mean(stage_likelihood)
            if average_likelihood == 0:
                print(f'averge likelihood is zero')
                log_average_likelihood = np.log(average_likelihood + 1e-20)
            else:
                log_average_likelihood = np.log(average_likelihood)
            all_participant_likelihood += log_average_likelihood

        # if all_participant_likelihood == 0:
        #     print("all_participant_likelihood is zero")
            
        # if np.any(participant_stages == 0):
        #     print("0 is in participant stages")
        # else:
        #     print("0 is not in participant stages")
        if  all_participant_likelihood > best_likelihood:
            best_likelihood = all_participant_likelihood 
            best_order = new_order
        else: 
            ratio = all_participant_likelihood/best_likelihood
            random_number = np.random.rand()
            if random_number < ratio:
                best_likelihood = all_participant_likelihood
                best_order = new_order
        print(f"iteration {_ + 1} done")
    return best_order, best_likelihood


In [24]:
# best_order, best_likelihood = metropolis_hastings_unknown_theta_phi(data_we_have, iterations = 20)

In [25]:
# best_order, np.array(list(real_ordering_dic.values()))